# Setups

In [1]:
import bw2data as bd
import bw2calc as bc
import numpy as np
from copy import deepcopy
from pathlib import Path
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import stats_arrays as sa
from scipy.stats import spearmanr

from gsa_framework.models.life_cycle_assessment import LCAModelBase
from gsa_framework.sensitivity_analysis.correlations import Correlations
from gsa_framework.sensitivity_analysis.saltelli_sobol import SaltelliSobol
from gsa_framework.utils import read_pickle, write_pickle, write_hdf5_array
from setups_paper_gwp import setup_corr, setup_lca_model_protocol
# from gsa_framework.utils import read_hdf5_array, write_hdf5_array
# from gsa_framework.visualization.plotting import plot_correlation_Y1_Y2, plot_histogram_Y1_Y2

from decimal import Decimal

In [2]:
path_base = Path('/data/user/kim_a')

# LCA model
bd.projects.set_current("GSA for protocol")
co = bd.Database("CH consumption 1.0")
demand_act = [act for act in co if "Food" in act["name"]]
assert len(demand_act) == 1
demand_act = demand_act[0]
demand = {demand_act: 1}
method = ("IPCC 2013", "climate change", "GWP 100a", "uncertain")

num_params = 20000
iter_corr = 4*num_params
gsa_corr = setup_corr(num_params, iter_corr, setup_lca_model_protocol, path_base)
S = gsa_corr.perform_gsa()
spearman = S['spearman']
inf_sorted = np.argsort(np.abs(spearman))[::-1]

model, write_dir, gsa_seed = setup_lca_model_protocol(
    path_base,
    num_params=None,
    write_dir=None,
)

path_lsa = model.write_dir / "LSA_scores"
path_lsa_include_inds_bio = path_lsa / "include_inds_bio.pickle"
include_inds_bio = read_pickle(path_lsa_include_inds_bio)
path_lsa_include_inds_cf = path_lsa / "include_inds_cf.pickle"
include_inds_cf = read_pickle(path_lsa_include_inds_cf)

include_inds_tech_forX = np.arange(model.uncertain_exchange_lengths['tech'])
include_inds_bio_forX  = model.uncertain_exchange_lengths['tech'] + include_inds_bio
include_inds_cf_forX   = model.uncertain_exchange_lengths['tech'] + \
                         model.uncertain_exchange_lengths['bio']  + include_inds_cf
parameter_choice_rm_noninf = np.hstack(
    [include_inds_tech_forX, include_inds_bio_forX, include_inds_cf_forX]
)

scores_dict = model.get_lsa_scores_pickle(model.write_dir / "LSA_scores", model.uncertain_exchanges_types)
num_params_lsa = 20000 #10000
where_high_var = model.get_where_high_var(scores_dict, num_params_lsa)
parameter_choice_rm_lowinf = parameter_choice_rm_noninf[where_high_var]

# num_params_ranking = 200
# gsa_seed_ranking = 555666
# iterations_ranking = num_params_ranking * 1600
# parameter_choice_inf = parameter_choice_rm_lowinf[inf_sorted[:num_params_ranking]]
# parameter_choice_inf = np.sort(parameter_choice_inf)

Unitcube samples ->    0.000 s
Rescaled samples ->    0.001 s
Model outputs    ->    0.000 s
GSA indices      ->    0.000 s
Total GSA time   ->    0.001 s 

LCA score is 211.57670115973556
Total number of uncertain exchanges is 408722
   tech=186602, bio=222049, cf=71


In [3]:
# len(parameter_choice_inf)

# Graph traversal screening

In [4]:
import numpy as np
import bw2data as bd
import bw2calc as bc

from dev.utils_graph_traversal import \
    filter_uncertain_technosphere_exchanges, \
    filter_uncertain_biosphere_exchanges, \
    filter_uncertain_characterization_exchanges, \
    collect_uncertain_exchanges

bd.projects.set_current("GSA for protocol")
co = bd.Database("CH consumption 1.0")
demand_act = [act for act in co if "Food" in act["name"]]
assert len(demand_act) == 1
demand_act = demand_act[0]
demand = {demand_act: 1}
method = ("IPCC 2013", "climate change", "GWP 100a", "uncertain")
lca = bc.LCA(demand, method)
lca.lci()
lca.lcia()

def graph_traversal_screening(lca, model, write_dir, cutoff, num_params_ranking):
    
    cutoff_str = '%.2E' % Decimal(cutoff)
    filename = "cutoff{}.params{}.pickle".format(cutoff_str, num_params_ranking)
    filepath = write_dir / filename
    
    if filepath.exists():
        data = read_pickle(filepath)
    else:
        exchanges_dict_filename = "exchanges_dict.cutoff{}.pickle".format(cutoff_str)
        exchanges_dict_filepath = write_dir / exchanges_dict_filename
        if exchanges_dict_filepath.exists():
            exchanges_dict = read_pickle(exchanges_dict_filepath)
            tech_inds_uncertain = exchanges_dict['tech']
            bio_inds_uncertain = exchanges_dict['bio']
            cf_inds_uncertain = exchanges_dict['cf']
        else:
            # tech_inds = filter_technosphere_exchanges(lca, cutoff)
            tech_inds_uncertain = filter_uncertain_technosphere_exchanges(lca, cutoff, 1e8)
            # bio_inds  = filter_biosphere_exchanges(lca, cutoff)
            bio_inds_uncertain = filter_uncertain_biosphere_exchanges(lca, cutoff)
            # ch_inds = filter_characterization_exchanges(lca, cutoff)
            cf_inds_uncertain = filter_uncertain_characterization_exchanges(lca, cutoff)
            exchanges_dict = {
                'tech': tech_inds_uncertain,
                'bio': bio_inds_uncertain,
                'cf': cf_inds_uncertain,
            }
            write_pickle(exchanges_dict, exchanges_dict_filepath)

        num_params_all = len(tech_inds_uncertain) + len(bio_inds_uncertain) + len(cf_inds_uncertain)
        subset_exchanges_dict_all = collect_uncertain_exchanges(exchanges_dict, num_params_all)
        
        subset_exchanges_dict = collect_uncertain_exchanges(exchanges_dict, num_params_ranking)

        row_ind = 0
        col_ind = 1
        sdicts = {
            'ranking': subset_exchanges_dict,
            'all': subset_exchanges_dict_all,
        }
        data = {}
        for sname, sdict in sdicts.items():
            parameter_choice_dict = {}
            for uncertain_exchange_type, exc_list in sdict.items():
                params = model.uncertain_params[uncertain_exchange_type]
                where_list = []
                if uncertain_exchange_type != 'cf':
                    for exc in exc_list:
                        where = np.where(np.logical_and(
                            params['row'] == exc[row_ind],
                            params['col'] == exc[col_ind],
                        ))[0]
                        assert len(where)==1
                        where_list.append(where[0])
                else:
                    for exc in exc_list:
                        where = np.where(params['row'] == exc[row_ind])[0]
                        assert len(where) == 1
                        where_list.append(where[0])
                where_list = sorted(where_list)
                parameter_choice_dict[uncertain_exchange_type] = np.array(where_list)

            parameter_choice_inf_tech = np.array(parameter_choice_dict['tech'])
            parameter_choice_inf_bio  = np.array(parameter_choice_dict['bio']) + model.uncertain_exchange_lengths['tech']
            parameter_choice_inf_cf   = np.array(parameter_choice_dict['cf']) + model.uncertain_exchange_lengths['tech'] + model.uncertain_exchange_lengths['bio']

            parameter_choice_inf_graph = np.hstack(
                [parameter_choice_inf_tech, parameter_choice_inf_bio, parameter_choice_inf_cf]
            )
            data[sname] = {
                "parameter_choice_dict": parameter_choice_dict, 
                "parameter_choice_inf_graph": parameter_choice_inf_graph
            }
            write_pickle(data, filepath)
    return data

In [21]:
write_dir_sct = write_dir / 'supply_chain'

data_all = {}
# percentage of the total score, cutoff=0.005 means 0.5 percent
cutoffs = np.array([1e-2, 5e-3, 1e-3, 5e-4, 1e-4, 5e-5, 1e-5])
nums_params_ranking = np.array([100,200,400,800,1600])
cutoffs, nums_params_ranking

(array([1.e-02, 5.e-03, 1.e-03, 5.e-04, 1.e-04, 5.e-05, 1.e-05]),
 array([ 100,  200,  400,  800, 1600]))

In [22]:
overlaps = {}
overlaps_all = {}
overlaps_ran = {}
for cutoff in cutoffs:
    print("--> {}".format(cutoff))
    data_all[cutoff] = {}
    overlaps[cutoff] = {}
    overlaps_all[cutoff] = {}
    overlaps_ran[cutoff] = {}
    for num_params_ranking in nums_params_ranking:
#         print(num_params_ranking)
        data_current = graph_traversal_screening(lca, model, write_dir_sct, cutoff, num_params_ranking)
        data_all[cutoff][num_params_ranking] = data_current
        
        parameter_choice_inf_lsa = parameter_choice_rm_lowinf[inf_sorted[:num_params_ranking]]
        parameter_choice_inf_lsa = np.sort(parameter_choice_inf_lsa)
        
        parameter_choice_inf_graph_all = data_current['all']['parameter_choice_inf_graph']
        parameter_choice_inf_graph_ran = data_current['ranking']['parameter_choice_inf_graph']
        
        overlap1 = np.intersect1d(parameter_choice_inf_lsa, parameter_choice_inf_graph_all)
        overlap2 = np.intersect1d(parameter_choice_inf_lsa, parameter_choice_inf_graph_ran)
        
        parameter_choice_inf_graph_all_tech = data_current['all']['parameter_choice_dict']['tech']
        parameter_choice_inf_graph_all_bio  = data_current['all']['parameter_choice_dict']['bio']
        parameter_choice_inf_graph_all_cf   = data_current['all']['parameter_choice_dict']['cf']
        parameter_choice_inf_graph_ran_tech = data_current['ranking']['parameter_choice_dict']['tech']
        parameter_choice_inf_graph_ran_bio  = data_current['ranking']['parameter_choice_dict']['bio']
        parameter_choice_inf_graph_ran_cf   = data_current['ranking']['parameter_choice_dict']['cf']
        
        parameter_choice_inf_lsa_tech = parameter_choice_inf_lsa[
            parameter_choice_inf_lsa<model.uncertain_exchange_lengths['tech']
        ]
        
        parameter_choice_inf_lsa_bio  = parameter_choice_inf_lsa[
            np.logical_and(
                parameter_choice_inf_lsa >= model.uncertain_exchange_lengths['tech'],
                parameter_choice_inf_lsa < model.uncertain_exchange_lengths['bio']+model.uncertain_exchange_lengths['tech'],
            )  
        ] - model.uncertain_exchange_lengths['tech']
        
        parameter_choice_inf_lsa_cf = parameter_choice_inf_lsa[
            parameter_choice_inf_lsa >= model.uncertain_exchange_lengths['bio']+model.uncertain_exchange_lengths['tech'],
        ] - model.uncertain_exchange_lengths['tech'] - model.uncertain_exchange_lengths['bio']
        
        overlap_all_tech = np.intersect1d(parameter_choice_inf_lsa_tech, parameter_choice_inf_graph_all_tech)
        overlap_all_bio  = np.intersect1d(parameter_choice_inf_lsa_bio,  parameter_choice_inf_graph_all_bio)
        overlap_all_cf   = np.intersect1d(parameter_choice_inf_lsa_cf,   parameter_choice_inf_graph_all_cf)
        
        overlap_ran_tech = np.intersect1d(parameter_choice_inf_lsa_tech, parameter_choice_inf_graph_ran_tech)
        overlap_ran_bio  = np.intersect1d(parameter_choice_inf_lsa_bio,  parameter_choice_inf_graph_ran_bio)
        overlap_ran_cf   = np.intersect1d(parameter_choice_inf_lsa_cf,   parameter_choice_inf_graph_ran_cf)
        
        overlaps[cutoff][num_params_ranking] = (len(parameter_choice_inf_graph_all), len(overlap1), len(overlap2))
        overlaps_all[cutoff][num_params_ranking] = (
            len(overlap_all_tech), len(overlap_all_bio), len(overlap_all_cf)
        )
        overlaps_ran[cutoff][num_params_ranking] = (
            len(overlap_ran_tech), len(overlap_ran_bio), len(overlap_ran_cf)
        )

--> 0.01
Impact threshold 0.000
128 exchanges selected
Impact threshold 0.033
100 exchanges selected
--> 0.005
Impact threshold 0.000
399 exchanges selected
Impact threshold 0.582
100 exchanges selected
--> 0.001
Impact threshold -0.699
3217 exchanges selected
Impact threshold 0.916
100 exchanges selected
--> 0.0005
Impact threshold -0.699
6134 exchanges selected
Impact threshold 0.916
100 exchanges selected
--> 0.0001
Impact threshold -0.699
17568 exchanges selected
Impact threshold 0.916
100 exchanges selected
--> 5e-05
TECHNOSPHERE (19353, 19353) filtering resulted in 23788 of 236414 UNCERTAIN exchanges and took 47965 iterations in 329.22 seconds.
BIOSPHERE (2102, 19353) filtering resulted in 829 of 16939 exchanges (88.96% of total impact) and took 1.61 seconds.
CHARACTERIZATION (2102,) filtering resulted in 19 of (2102,) exchanges (46.31% of total impact) and took 0.0 seconds.
Impact threshold -0.699
24636 exchanges selected
Impact threshold 0.916
100 exchanges selected
Impact thre

In [23]:
import pandas as pd

In [24]:
pd.DataFrame.from_dict(overlaps)

,0.01000,0.00500,0.00100,0.00050,0.00010,0.00005,0.00001
100,"(128, 46, 46)","(399, 62, 62)","(3217, 78, 63)","(6134, 78, 63)","(17568, 82, 63)","(24636, 89, 63)","(43660, 97, 63)"
200,"(128, 53, 53)","(399, 78, 78)","(3217, 128, 101)","(6134, 139, 101)","(17568, 157, 101)","(24636, 167, 101)","(43660, 188, 101)"
400,"(128, 54, 54)","(399, 87, 87)","(3217, 169, 129)","(6134, 214, 129)","(17568, 279, 129)","(24636, 305, 129)","(43660, 361, 129)"
800,"(128, 58, 58)","(399, 101, 101)","(3217, 247, 179)","(6134, 327, 190)","(17568, 509, 190)","(24636, 578, 190)","(43660, 705, 190)"
1600,"(128, 61, 61)","(399, 118, 118)","(3217, 379, 287)","(6134, 548, 303)","(17568, 974, 310)","(24636, 1109, 310)","(43660, 1396, 310)"


In [25]:
pd.DataFrame.from_dict(overlaps_all)

,0.01000,0.00500,0.00100,0.00050,0.00010,0.00005,0.00001
100,"(30, 12, 4)","(42, 15, 5)","(53, 20, 5)","(53, 20, 5)","(55, 22, 5)","(56, 28, 5)","(57, 35, 5)"
200,"(36, 13, 4)","(53, 20, 5)","(87, 32, 9)","(96, 34, 9)","(109, 39, 9)","(112, 45, 10)","(120, 58, 10)"
400,"(37, 13, 4)","(61, 21, 5)","(125, 35, 9)","(166, 39, 9)","(225, 45, 9)","(243, 52, 10)","(275, 76, 10)"
800,"(41, 13, 4)","(73, 23, 5)","(184, 54, 9)","(257, 61, 9)","(427, 73, 9)","(483, 85, 10)","(575, 120, 10)"
1600,"(44, 13, 4)","(90, 23, 5)","(303, 66, 10)","(459, 79, 10)","(862, 102, 10)","(979, 119, 11)","(1204, 181, 11)"


In [26]:
pd.DataFrame.from_dict(overlaps_ran)

,0.01000,0.00500,0.00100,0.00050,0.00010,0.00005,0.00001
100,"(30, 12, 4)","(42, 15, 5)","(42, 16, 5)","(42, 16, 5)","(42, 16, 5)","(42, 16, 5)","(42, 16, 5)"
200,"(36, 13, 4)","(53, 20, 5)","(65, 28, 8)","(65, 28, 8)","(65, 28, 8)","(65, 28, 8)","(65, 28, 8)"
400,"(37, 13, 4)","(61, 21, 5)","(86, 34, 9)","(86, 34, 9)","(86, 34, 9)","(86, 34, 9)","(86, 34, 9)"
800,"(41, 13, 4)","(73, 23, 5)","(117, 53, 9)","(121, 60, 9)","(121, 60, 9)","(121, 60, 9)","(121, 60, 9)"
1600,"(44, 13, 4)","(90, 23, 5)","(212, 65, 10)","(215, 78, 10)","(211, 89, 10)","(211, 89, 10)","(211, 89, 10)"


In [57]:
data_current['ranking']['parameter_choice_dict']['tech'].shape

(1143,)

In [60]:
parameter_choice_inf_lsa_tech.shape

(1322,)

In [46]:
(1322 - 211)/21.1

52.654028436018955

In [35]:
54.17*21.1


1142.987

In [61]:
1322/42.2 - 5

26.327014218009477

In [58]:
211/5

42.2

In [ ]:
a = parameter_choice_inf[parameter_choice_inf<model.uncertain_exchange_lengths['tech']]
b = parameter_choice_inf_tech
a.shape, b.shape, np.intersect1d(a,b).shape

# Compare our screening with graph traversal

In [ ]:
parameter_choice_inf_intersect = np.intersect1d(parameter_choice_inf, parameter_choice_inf_graph)
len(parameter_choice_inf_intersect)

In [ ]:
%%time
from gsa_framework.convergence_robustness_validation import Validation

default_uncertain_amounts = np.hstack([
    v for v in model.default_uncertain_amounts.values()
])

iterations_validation = 300
validation_seed = 100023423
lca_scores_axis_title = r"$\text{LCA scores, [kg CO}_2\text{-eq}]$"
if __name__ == "__main__":
    val = Validation(
        model=model,
        iterations=iterations_validation,
        seed=validation_seed,
        default_x_rescaled=default_uncertain_amounts,
        write_dir=write_dir,
        model_output_name=lca_scores_axis_title,
    )    
    

In [ ]:
if __name__ == "__main__":
    tag = "SpearmanIndex"
    Y_subset = val.get_influential_Y_from_parameter_choice(influential_inputs=parameter_choice_inf, tag=tag)

fig=val.plot_correlation_Y_all_Y_inf(Y_subset, num_influential=parameter_choice_inf.shape[0], tag=tag)
fig.show()

fig=val.plot_histogram_Y_all_Y_inf(Y_subset, num_influential=parameter_choice_inf.shape[0], tag=tag)
fig.show()

In [ ]:
len(parameter_choice_inf_graph)

In [ ]:
if __name__ == "__main__":
    tag = "GraphTraversal"
    Y_subset = val.get_influential_Y_from_parameter_choice(influential_inputs=parameter_choice_inf_graph, tag=tag)

fig=val.plot_correlation_Y_all_Y_inf(Y_subset, num_influential=parameter_choice_inf_graph.shape[0], tag=tag)
fig.show()

fig=val.plot_histogram_Y_all_Y_inf(Y_subset, num_influential=parameter_choice_inf_graph.shape[0], tag=tag)
fig.show()

In [ ]:
if __name__ == "__main__":
    tag = "SpearmanGraphIntersection"
    Y_subset = val.get_influential_Y_from_parameter_choice(influential_inputs=parameter_choice_inf_intersect, tag=tag)

fig=val.plot_correlation_Y_all_Y_inf(Y_subset, num_influential=parameter_choice_inf_intersect.shape[0], tag=tag)
fig.show()

fig=val.plot_histogram_Y_all_Y_inf(Y_subset, num_influential=parameter_choice_inf_intersect.shape[0], tag=tag)
fig.show()

# GSA for prioritized list

In [ ]:
option = "protocol"
if option == "graph":
    parameter_choice_inf_use = parameter_choice_inf_graph
elif option == "protocol":
    parameter_choice_inf_use = parameter_choice_inf

parameter_choice_inf_tech = parameter_choice_inf_use[
    parameter_choice_inf_use < model.uncertain_exchange_lengths['tech']
]
tech_inds = parameter_choice_inf_tech
tech_params = model.uncertain_params['tech'][tech_inds]
tech_where = model.uncertain_params_selected_where_dict['tech'][tech_inds]

parameter_choice_inf_bio = parameter_choice_inf_use[
    np.logical_and(
        parameter_choice_inf_use >= model.uncertain_exchange_lengths['tech'],
        parameter_choice_inf_use < model.uncertain_exchange_lengths['bio']+model.uncertain_exchange_lengths['tech'],
    )  
]
bio_inds = parameter_choice_inf_bio-model.uncertain_exchange_lengths['tech']
bio_params = model.uncertain_params['bio'][bio_inds]
bio_where = model.uncertain_params_selected_where_dict['bio'][bio_inds]

parameter_choice_inf_cf = parameter_choice_inf_use[
    parameter_choice_inf_use >= model.uncertain_exchange_lengths['bio']+model.uncertain_exchange_lengths['tech'],
]
cf_inds = parameter_choice_inf_cf-model.uncertain_exchange_lengths['tech']-model.uncertain_exchange_lengths['bio']
cf_params = model.uncertain_params['cf'][cf_inds]
cf_where = model.uncertain_params_selected_where_dict['cf'][cf_inds]

uncertain_params = {
    'tech': tech_params,
    'bio': bio_params,
    'cf': cf_params,
}
uncertain_params_selected_where_dict = {
    'tech': tech_where,
    'bio': bio_where,
    'cf': cf_where,    
}

In [ ]:
if __name__ == "__main__":    
    model_ranking = LCAModelBase(
        demand, 
        method, 
        uncertain_params, 
        uncertain_params_selected_where_dict,
    )
    gsa = SaltelliSobol(iterations=iterations_ranking, model=model_ranking, write_dir=write_dir)
    S_salt = gsa.perform_gsa()
    fig = gsa.plot_sa_results(S_salt)

In [ ]:
filename_Y = "{}.{}{}".format(gsa.filepath_Y.stem, option, gsa.filepath_Y.suffix)
filepath_Y = gsa.filepath_Y.parent / filename_Y
filepath_Y

In [ ]:
# %%time
# X = gsa.generate_unitcube_samples(iterations_ranking)
# Xr = model_ranking.rescale(X)
# Y = model_ranking(Xr)
# write_hdf5_array(Y, filepath_Y)

# Convergence, robustness

In [ ]:
from gsa_framework.sensitivity_methods.saltelli_sobol import sobol_indices_stability
from gsa_framework.convergence_robustness_validation import Convergence, Robustness
from gsa_framework.utils import read_hdf5_array, read_pickle, write_pickle

num_bootstrap = 100
num_steps = 50

# Convergence class
conv = Convergence(
    gsa.filepath_Y,
    gsa.num_params,
    gsa.generate_gsa_indices,
    gsa.gsa_label,
    gsa.write_dir,
    num_steps=num_steps,
)
np.random.seed(gsa.seed)
stability_seeds = np.random.randint(
    low=0,
    high=2147483647,
    size=(len(conv.iterations_for_convergence), num_bootstrap),
)

filename_S = "stability.S.{}.{}.{}Step{}.{}.{}.pickle".format(
    gsa.gsa_label,
    gsa.sampling_label,
    gsa.iterations,
    conv.iterations_step,
    num_bootstrap,
    gsa.seed,
)
filepath_S = gsa.write_dir / "arrays" / filename_S
if filepath_S.exists():
    print("--> {} already exists".format(filename_S))
    S_dict_stability = read_pickle(filepath_S)
else:
    Y = read_hdf5_array(gsa.filepath_Y).flatten()
    S_dict_stability = sobol_indices_stability(
        Y,
        gsa.num_params,
        conv.iterations_for_convergence,
        num_bootstrap,
        stability_seeds,
    )
    write_pickle(S_dict_stability, filepath_S)

### 1. For convergence, total order should be higher than first order, first order should be non-negative

In [ ]:
x = np.arange(num_params_ranking)

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=x,
        y=S_salt['First order'],
        mode="markers",
        name="First order",
        marker=dict(
            color="blue",
        ),
        showlegend=True,
    ),
)

fig.add_trace(
    go.Scatter(
        x=x,
        y=S_salt['Total order'],
        mode="markers",
        name="Total order",
        marker=dict(
            color="red",
            symbol="x",
        ),
        showlegend=True,
    ),
)
fig.update_xaxes(title='Model inputs')
fig.update_yaxes(title='Sobol indices')

### 2. Confidence intervals at last step for all inputs

In [ ]:
from dev.utils_paper_plotting import *

num_ranks = 20
bootstrap_ranking_tag = 'paper2'
stability_dicts = [S_dict_stability]
st = Robustness(
    stability_dicts,
    write_dir,
    num_ranks=num_ranks,
    bootstrap_ranking_tag=bootstrap_ranking_tag,
    num_params_screening=int(0.90 * gsa.num_params),
)

In [ ]:
x = np.arange(num_params_ranking)

sa_names = ['first', 'total']
colors = {
    'first': color_blue_tuple, 
    'total': color_blue_tuple,
}
opacity = 0.65

fig = make_subplots(
    rows=2,
    cols=1,
    shared_xaxes=True,
    subplot_titles = sa_names,
)

col = 1

for row, sa_name in enumerate(sa_names):
    row += 1
    color = colors[sa_name]
    if sa_name == 'first':
        y = S_salt['First order']
    elif sa_name ==  'total':
        y = S_salt['Total order']
    width = st.confidence_intervals[sa_name][-1,:]
    fig.add_trace(
        go.Scatter(
            x=x,
            y=y,
            mode="markers",
            name=sa_name,
            marker=dict(
                color=color_orange_rgb,
            ),
            showlegend=False,
            error_y=dict(
                type='data', # value of error bar given in data coordinates
                symmetric=False,
                array=width/2,
                arrayminus=width/2,
                visible=True,
                color=color_blue_rgb,
            )
        ),
        row=row,
        col=col
    )

fig.update_xaxes(title='Model inputs')
fig.update_yaxes(title='Sobol indices')

### 3. Max confidence invervals at all steps, $Stat_{indices}$

In [ ]:
opacity = 0.65

fig = make_subplots(
    rows=2,
    cols=1,
    shared_xaxes=True,
    subplot_titles = sa_names,
)

col = 1
for row, sa_name in enumerate(sa_names):
    row += 1
    showlegend = False
    color = colors[sa_name]
    x = st.iterations[sa_name]
    y = np.zeros(len(x))
    width = st.confidence_intervals_max[sa_name]
    lower = y - width / 2
    upper = y + width / 2
    fig.add_trace(
        go.Scatter(
            x=x,
            y=y,
            mode="lines",
            opacity=1,
            showlegend=showlegend,
            name = sa_name,
            marker=dict(
                color="rgba({},{},{},{})".format(
                    color[0],
                    color[1],
                    color[2],
                    1,
                ),
            ),
        ),
        row=row,
        col=col,
    )
    showlegend = False
    fig.add_trace(
        go.Scatter(
            x=x,
            y=lower,
            mode="lines",
            opacity=opacity,
            showlegend=False,
            marker=dict(
                color="rgba({},{},{},{})".format(
                    color[0],
                    color[1],
                    color[2],
                    opacity,
                ),
            ),
            line=dict(width=0),
        ),
        row=row,
        col=col,
    )
    fig.add_trace(
        go.Scatter(
            x=x,
            y=upper,
            showlegend=False,
            line=dict(width=0),
            mode="lines",
            fillcolor="rgba({},{},{},{})".format(
                color[0],
                color[1],
                color[2],
                opacity,
            ),
            fill="tonexty",
        ),
        row=row,
        col=col,
    )
fig.update_xaxes(title='MC iterations')
fig.update_yaxes(title='Max confidence interval')

# Prioritized list

In [ ]:
total = S_salt['Total order']
total_argsort = np.argsort(total)[::-1]
total_sorted = total[total_argsort]
first_sorted = S_salt['First order'][total_argsort]
width = st.confidence_intervals['total'][-1,:]
width_sorted = width[total_argsort]

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=np.arange(len(total_sorted)),
        y=total_sorted,
        mode="markers",
        name="Total order",
        showlegend=False,
        marker_color = color_orange_rgb,
        error_y=dict(
            type='data', # value of error bar given in data coordinates
            symmetric=False,
            array=width_sorted/2,
            arrayminus=width_sorted/2,
            visible=True,
            color=color_blue_rgb,
        )
    ),
)
fig.update_xaxes(title='Model inputs')
fig.update_yaxes(title='Total order indices')

In [ ]:
num_ranked = 23

parameter_choice_ranked = parameter_choice_inf[total_argsort][:num_ranked]

len_tech = model.uncertain_exchange_lengths['tech']
where_tech = np.where(parameter_choice_ranked<len_tech)[0]
ind_tech = parameter_choice_ranked[where_tech]
inf_tech_params = {
    where_tech[i]: {
        "type": "tech",
        "param": model.uncertain_params['tech'][ind_tech[i]]
    }
    for i in range(len(where_tech))
}

len_bio = model.uncertain_exchange_lengths['bio']
where_bio = np.where(np.logical_and(
    parameter_choice_ranked>=len_tech,
    parameter_choice_ranked<len_tech+len_bio,
))[0]
ind_bio = parameter_choice_ranked[where_bio]-len_tech
inf_bio_params = {
    where_bio[i]: {
        "type": "bio",
        "param": model.uncertain_params['bio'][ind_bio[i]]
    }
    for i in range(len(where_bio))
}

len_cf = model.uncertain_exchange_lengths['cf']
where_cf = np.where(np.logical_and(
    parameter_choice_ranked>=len_tech+len_bio,
    parameter_choice_ranked<len_tech+len_bio+len_cf,
))[0]
ind_cf = parameter_choice_ranked[where_cf]-len_tech-len_bio
inf_cf_params = {
    where_cf[i]: {
        "type": "cf",
        "param": model.uncertain_params['cf'][ind_cf[i]]
    }
    for i in range(len(where_cf))
}

params = {**inf_tech_params, **inf_bio_params, **inf_cf_params}
params = {k : params[k] for k in sorted(params)}

TECH_IND = 0
BIO_IND = 2
distributions = {
    sa.NormalUncertainty.id: 'normal',
    sa.LognormalUncertainty.id: 'lognml',
    sa.UniformUncertainty.id: 'unifrm',
}

for rank, dict_ in params.items():
    exchange_type = dict_['type']
    param = dict_['param']
    row = param['row']
    col = param['col']
    print(
        "{:2d}. total={:5.3f}, {}, amount={:8.5f}, scale={:5.3f}".format(
            rank, 
            total_sorted[rank],
            distributions[param['uncertainty_type']],
            param['amount'],
            param['scale'],
        )
    )      
    if exchange_type=='tech':
        act_in = bd.get_activity(model.lca.reverse_dict()[TECH_IND][row])
        act_out = bd.get_activity(model.lca.reverse_dict()[TECH_IND][col])
        print("act out:    {}, {}".format(act_out['name'], act_out['location']))
        print("act  in:    {}, {}, {} \n".format(act_in['name'], act_in['unit'], act_in['location']))
    elif exchange_type=='bio':
        act_in = bd.get_activity(model.lca.reverse_dict()[BIO_IND][row])
        act_out = bd.get_activity(model.lca.reverse_dict()[TECH_IND][col])
        print("act out:    {}, {}".format(act_out['name'], act_out['location']))
        print("act  in:    {}, {} \n".format(act_in['name'], act_in['unit']))
    elif exchange_type=='cf':
        act_in = bd.get_activity(model.lca.reverse_dict()[BIO_IND][row])
        print("GWP of:    {} \n".format(act_in['name'])) 

In [ ]:
num_ranked = 24
parameter_choice_ranked = parameter_choice_inf[total_argsort][:num_ranked]

if __name__ == "__main__":
    tag = "TotalRanked.graph"
    Y_subset = val.get_influential_Y_from_parameter_choice(influential_inputs=parameter_choice_ranked, tag=tag)

fig=val.plot_correlation_Y_all_Y_inf(Y_subset, num_influential=parameter_choice_ranked.shape[0], tag=tag)
fig.show()

fig=val.plot_histogram_Y_all_Y_inf(Y_subset, num_influential=parameter_choice_ranked.shape[0], tag=tag)
fig.show()

In [ ]:
from gsa_framework.visualization.plotting import *

def plot_hist_val(Y_subset, parameter_choice_ranked):
    bin_min = min(val.Y_all)
    bin_max = max(val.Y_all)
    fig = plot_histogram_Y1_Y2(
        val.Y_all,
        Y_subset,
        bin_min=bin_min,
        bin_max=bin_max,
        num_bins=60,
        trace_name1="All parameters vary",
        trace_name2="Only influential vary",
        color1="#636EFA",
        color2="#EF553B",
        opacity=0.65,
        xaxes_title_text=val.model_output_name,
        showtitle=True,
    )
    fig.update_yaxes(range=[0,25])
    fig.write_image(
        val.create_figure_Y_all_Y_inf_histogram_filepath(
            parameter_choice_ranked.shape[0], tag, "pdf"
        ).as_posix()
    )

In [ ]:
%%time
fig_format = ["pdf"]
spearman_yy = []
num_ranked_max = 25
num_ranked_arr = np.hstack(
    [
        np.arange(1,10),
        np.arange(10,num_ranked_max,2)
    ]
)
tag = "TotalRanked.{}".format("graph")
if __name__ == "__main__":
    for num_ranked in num_ranked_arr:
        print(num_ranked)
        parameter_choice_ranked = parameter_choice_inf[total_argsort][:num_ranked]
        Y_subset = val.get_influential_Y_from_parameter_choice(influential_inputs=parameter_choice_ranked, tag=tag)
        s, _ = spearmanr(val.Y_all, Y_subset)
        spearman_yy.append(s)
        fig=val.plot_correlation_Y_all_Y_inf(
            Y_subset, num_influential=parameter_choice_ranked.shape[0], tag=tag, fig_format=fig_format,
        )
        plot_hist_val(Y_subset, parameter_choice_ranked)
        
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=num_ranked_arr,
        y=spearman_yy,
        mode="markers+lines",
        showlegend=False,
        marker_color = color_blue_rgb,
    ),
)
fig.update_xaxes(title='Number of varying influential inputs')
fig.update_yaxes(title='Spearman correlation between Y_all and Y_inf')

In [ ]:
%%time
fig_format = ["pdf"]
spearman_yy = []
num_ranked_max = 25
num_ranked_arr = np.hstack(
    [
        np.arange(1,10),
        np.arange(10,num_ranked_max,2)
    ]
)
tag = "TotalRanked.{}".format("protocol")
if __name__ == "__main__":
    for num_ranked in num_ranked_arr:
        print(num_ranked)
        parameter_choice_ranked = parameter_choice_inf[total_argsort][:num_ranked]
        Y_subset = val.get_influential_Y_from_parameter_choice(influential_inputs=parameter_choice_ranked, tag=tag)
        s, _ = spearmanr(val.Y_all, Y_subset)
        spearman_yy.append(s)
        fig=val.plot_correlation_Y_all_Y_inf(
            Y_subset, num_influential=parameter_choice_ranked.shape[0], tag=tag, fig_format=fig_format,
        )
        plot_hist_val(Y_subset, parameter_choice_ranked)
        
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=num_ranked_arr,
        y=spearman_yy,
        mode="markers+lines",
        showlegend=False,
        marker_color = color_blue_rgb,
    ),
)
fig.update_xaxes(title='Number of varying influential inputs')
fig.update_yaxes(title='Spearman correlation between Y_all and Y_inf')

# Inverted validation plots

In [ ]:
%%time
num_ranked = 9
parameter_choice_ranked = parameter_choice_inf[total_argsort][:num_ranked]
parameter_choice_ranked_inv = np.setdiff1d(np.arange(model.num_params), parameter_choice_ranked)

if __name__ == "__main__":
    tag = "TotalRankedInv"
    Y_subset = val.get_influential_Y_from_parameter_choice(influential_inputs=parameter_choice_ranked_inv, tag=tag)

fig=val.plot_correlation_Y_all_Y_inf(Y_subset, num_influential=parameter_choice_ranked.shape[0], tag=tag)
fig.show()

fig=val.plot_histogram_Y_all_Y_inf(Y_subset, num_influential=parameter_choice_ranked.shape[0], tag=tag)
fig.show()

In [ ]:
%%time
num_ranked = 23
parameter_choice_ranked = parameter_choice_inf[total_argsort][:num_ranked]
parameter_choice_ranked_inv = np.setdiff1d(np.arange(model.num_params), parameter_choice_ranked)

if __name__ == "__main__":
    tag = "TotalRankedInv"
    Y_subset = val.get_influential_Y_from_parameter_choice(influential_inputs=parameter_choice_ranked_inv, tag=tag)

fig=val.plot_correlation_Y_all_Y_inf(Y_subset, num_influential=parameter_choice_ranked.shape[0], tag=tag)
fig.show()

fig=val.plot_histogram_Y_all_Y_inf(Y_subset, num_influential=parameter_choice_ranked.shape[0], tag=tag)
fig.show()